In [ ]:
import org.apache.spark.sql.Row
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions.unix_timestamp

println(sc.version)

In [ ]:
def mapRdd(df: DataFrame) = {
    val schema = df.schema.toList
    df.rdd.map(
        row => schema.map {
                field => field.name -> row.get(row.fieldIndex(field.name))
            }
    ).map(_.toMap).collect()
}   


val timestamp = unix_timestamp($"ts", "yyyy-MM-dd HH:mm:ss.SSS").cast("double").alias("timestamp")

In [ ]:
import org.apache.kudu.spark.kudu._
import sqlContext.implicits._

val table = "mowers"
val masters = List(
    "ip-10-0-0-118.us-west-2.compute.internal:7051",
    "ip-10-0-0-135.us-west-2.compute.internal:7051",
    "ip-10-0-0-136.us-west-2.compute.internal:7051"
).mkString(",")

val kuduContext = new KuduContext(masters)

kuduContext.tableExists(table)

In [ ]:
CREATE DATABASE xke;

In [ ]:
kuduContext.tableExists(table)

In [ ]:
-- TBLPROPERTIES(
--  'kudu.table_name' = 'mowers'
-- );

In [ ]:
SELECT * FROM xke.mowers LIMIT 50;

In [ ]:
-- wrong insert
INSERT INTO xke.mowers VALUES("foo", "2017-02-01 10:10:00.200", 0.0, 0.0, 0.0, false, 0.0);
SELECT * FROM xke.mowers WHERE host = "foo";

In [ ]:
val kuduOpt = Map("kudu.master" -> masters, "kudu.table" -> table)

In [ ]:
val df = sqlContext.read.options(kuduOpt).kudu

In [ ]:
df.printSchema

In [ ]:
val heats = (df.filter($"host" === "actor-mower-1")
    .select($"heat", timestamp)
    .sort($"timestamp".desc).limit(600)
    .sort($"timestamp"))
    
val res1 = mapRdd(heats)
%table res1

In [ ]:
val heatMean = df.filter($"cooling").filter($"elapsed" >= 20.0).groupBy($"host").mean("heat")
val res2 = mapRdd(heatMean)
%table res2

In [ ]:
UPDATE xke.mowers SET hoho = "Cloud" WHERE tag = "-1" AND ts >= '2017-02-01';